In [53]:
## Importing the required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from geopy.geocoders import Nominatim
import requests

In [47]:
address = 'Vancouver, Canada'
geolocator = Nominatim(user_agent="anduleh2")
location = geolocator.geocode(address)
vancity_latitude = location.latitude
vancity_longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,vancity_latitude, vancity_longitude))

The geograpical coordinate of Vancouver, Canada are 49.2608724, -123.1139529.


In [48]:
df = pd.read_csv('ftp://webftp.vancouver.ca/OpenData/csv/cov_localareas.csv')

In [49]:
clean_df = df.rename({'NAME':'Neighbourhood'}, axis='columns').drop('MAPID',axis=1)
clean_df['Latitude'] = ''
clean_df['Longitude'] = ''
for i, row in clean_df.iterrows():
    address_iter = '{}, Vancouver, Canada'.format(row['Neighbourhood'])
    location = geolocator.geocode(address_iter)
    clean_df.at[i,'Latitude'] = location.latitude
    clean_df.at[i,'Longitude'] =location.longitude

In [12]:
clean_df

,Neighbourhood,Latitude,Longitude
0,Sunset,49.2196,-123.09
1,Mount Pleasant,49.2633,-123.097
2,Riley Park,49.2474,-123.103
3,Downtown,49.2834,-123.117
4,Kitsilano,49.2694,-123.155
5,Dunbar-Southlands,49.2535,-123.185
6,Kerrisdale,49.2347,-123.155
7,Arbutus-Ridge,49.241,-123.167
8,West Point Grey,49.2645,-123.185
9,Marpole,49.2092,-123.136


In [13]:
map_vancouver = folium.Map(location=[vancity_latitude, vancity_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(clean_df['Latitude'], clean_df['Longitude'], clean_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_vancouver)  
    
map_vancouver

In [14]:
# @hidden_cell
CLIENT_ID = 'QGIGC40OIPFLAHA4SZRPLBFFNM1QXA4V2G2LQW5244VYY50P' # your Foursquare ID
CLIENT_SECRET = 'QSE3UMXWQFRONXR4SACUKZJ1HY3HMVYZM1IOT3B1EZXWN0YQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QGIGC40OIPFLAHA4SZRPLBFFNM1QXA4V2G2LQW5244VYY50P
CLIENT_SECRET:QSE3UMXWQFRONXR4SACUKZJ1HY3HMVYZM1IOT3B1EZXWN0YQ


### Lets check out the First Neighbourhood in the DataFrame

In [50]:
first_n = clean_df.loc[0].to_dict()
first_n_name = first_n['Neighbourhood']
first_n_lat = first_neighbourhood['Latitude']
first_n_lon = first_neighbourhood['Longitude']
print('The geograpical coordinate of {} are {}, {}.'.format(first_n['Neighbourhood'] + ', ' + address,first_n_lat, first_n_lon))

The geograpical coordinate of Sunset, Vancouver, Canada are 49.2633296, -123.0965885.


In [51]:
# FourSquare Query URL
LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_n_lat, 
    first_n_lon, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=QGIGC40OIPFLAHA4SZRPLBFFNM1QXA4V2G2LQW5244VYY50P&client_secret=QSE3UMXWQFRONXR4SACUKZJ1HY3HMVYZM1IOT3B1EZXWN0YQ&v=20180605&ll=49.2633296,-123.0965885&radius=500&limit=100'

In [54]:
results = requests.get(url).json()
results["response"]['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '50f1dd8ee4b0ee8fd93bbfff',
   'name': 'Dude Chilling Park',
   'location': {'address': 'E 8th Ave',
    'crossStreet': 'at Guelph Park',
    'lat': 49.26372990146711,
    'lng': -123.09679627418517,
    'labeledLatLngs': [{'label': 'display',
      'lat': 49.26372990146711,
      'lng': -123.09679627418517}],
    'distance': 47,
    'cc': 'CA',
    'city': 'Vancouver',
    'state': 'BC',
    'country': 'Canada',
    'formattedAddress': ['E 8th Ave (at Guelph Park)',
     'Vancouver BC',
     'Canada']},
   'categories': [{'id': '52e81612bcbc57f1066b79ed',
     'name': 'Outdoor Sculpture',
     'pluralName': 'Outdoor Sculptures',
     'shortName': 'Outdoor Sculpture',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/sculpture_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'c

In [55]:
# venues within 500m of Kitusuru
for row in results["response"]['groups'][0]['items']:
    print(row['venue']['categories'][0]['name'])

Outdoor Sculpture
Sushi Restaurant
Tea Room
Bookstore
Burrito Place
Sushi Restaurant
Brewery
Coffee Shop
Brewery
Bagel Shop
Chinese Restaurant
Diner
Lounge
Donut Shop
Sushi Restaurant
Pizza Place
Taco Place
Music Venue
Indian Restaurant
Grocery Store
Coffee Shop
Vietnamese Restaurant
Bar
Clothing Store
Food Truck
Indian Restaurant
Arts & Crafts Store
Lounge
Board Shop
Thrift / Vintage Store
Pub
Coffee Shop
Breakfast Spot
Sandwich Place
Coffee Shop
New American Restaurant
Pool Hall
Peruvian Restaurant
Ethiopian Restaurant
Record Shop
Gastropub
Library
Diner
Noodle House
Breakfast Spot
Arts & Crafts Store
Breakfast Spot
Vietnamese Restaurant
Pharmacy
Mexican Restaurant
Sandwich Place
Gym
Bistro
Diner
Sports Bar
Coffee Shop
Coffee Shop
Bakery
Hotel
Sandwich Place
Restaurant
Tapas Restaurant
Liquor Store
Sandwich Place
Thrift / Vintage Store
Café
Thrift / Vintage Store


In [57]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# get nearby venues
vancouver_venues = getNearbyVenues(names=clean_df["Neighbourhood"], 
                                 latitudes=clean_df["Latitude"], 
                                 longitudes=clean_df["Longitude"])

Sunset
Mount Pleasant
Riley Park
Downtown
Kitsilano
Dunbar-Southlands
Kerrisdale
Arbutus-Ridge
West Point Grey
Marpole
Oakridge
Shaughnessy
Fairview
South Cambie
West End
Killarney
Renfrew-Collingwood
Hastings-Sunrise
Victoria-Fraserview
Kensington-Cedar Cottage
Strathcona
Grandview-Woodland


In [61]:
map_vancity = folium.Map(location=[vancity_latitude, vancity_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(vancouver_venues['Venue Latitude'], vancouver_venues['Venue Longitude'], vancouver_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#00A86B',
        fill_opacity=0.7).add_to(map_vancity)  
    
map_vancity

In [73]:
# Importing clustering libraries
from sklearn.cluster import DBSCAN
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [76]:
coords = vancouver_venues[['Venue Latitude', 'Venue Longitude']].values

In [94]:
kms_per_radian = 10000
epsilon = 1.0 / kms_per_radian # Max distance points can be from each other
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

Number of clusters: 16


In [95]:
# Add the clusters to the nairobi_restaurants dataframe
vancouver_venues["cluster"] = cluster_labels
vancouver_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,cluster
0,Sunset,49.219593,-123.090239,Breka Bakery & Café,49.225172,-123.090856,Bakery,0
1,Sunset,49.219593,-123.090239,So Hyang Korean Cuisine,49.227019,-123.090842,Korean Restaurant,0
2,Sunset,49.219593,-123.090239,Shoom 雅菀食府,49.224861,-123.090948,Cantonese Restaurant,0
3,Sunset,49.219593,-123.090239,Panos Greek Taverna,49.210968,-123.092143,Restaurant,1
4,Sunset,49.219593,-123.090239,Deer Garden Signatures,49.227635,-123.090703,Chinese Restaurant,0
5,Sunset,49.219593,-123.090239,BAKER & TABLE Cafe,49.226324,-123.090678,Bakery,0
6,Sunset,49.219593,-123.090239,Shoppers Drug Mart,49.227302,-123.090720,Pharmacy,0
7,Sunset,49.219593,-123.090239,Ho Yuen Kee Restaurant,49.227979,-123.090478,Chinese Restaurant,0
8,Sunset,49.219593,-123.090239,Sunset Community Centre,49.222767,-123.101351,Event Space,0
9,Sunset,49.219593,-123.090239,Dollar Tree,49.225731,-123.090710,Discount Store,0


In [96]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [97]:
# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [ ]:
map_vancity_clusters = folium.Map(location=[vancity_latitude, vancity_longitude], zoom_start=11)

# add markers to map
markers_colors = []
for lat, lon, poi, cluster in zip(vancouver_venues['Venue Latitude'], vancouver_venues['Venue Longitude'], vancouver_venues['Venue'], vancouver_venues['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_vancity_clusters)
       
map_vancity_clusters